In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 62.7 MB/s eta 0:00:00


In [9]:
import json
import numpy as np
import faiss
import os
import sys
from sentence_transformers import SentenceTransformer
from typing import Tuple, List, Optional, Dict
from pathlib import Path

# ==============================================================================
# 1. BUILDER CLASS (Tạo Index từ dữ liệu Train)
# ==============================================================================

class R1IndexBuilder:
    """Builder để tạo FAISS index từ file jsonl training"""

    def __init__(
        self,
        input_file: str,
        output_dir: str,
        model_name: str = "keepitreal/vietnamese-sbert"
    ):
        self.input_file = Path(input_file)
        self.output_dir = Path(output_dir)
        self.model_name = model_name

        # Tạo thư mục output nếu chưa có
        self.output_dir.mkdir(parents=True, exist_ok=True)

        self.index_path = self.output_dir / "r1.index"
        self.meta_path = self.output_dir / "r1_metadata.json"

    def build(self, force_rebuild: bool = False) -> None:
        """Thực hiện build index"""

        if self.index_path.exists() and not force_rebuild:
            print(f"⚠️ Index đã tồn tại tại: {self.index_path}")
            return

        if not self.input_file.exists():
            raise FileNotFoundError(
                f"❌ Không tìm thấy file train: {self.input_file}\n"
            )

        print(f"🚀 Đang khởi tạo model embedding: {self.model_name}...")
        embedder = SentenceTransformer(self.model_name)

        print(f"📂 Đang đọc dữ liệu từ {self.input_file}...")
        sentences = []
        labels = []

        with open(self.input_file, 'r', encoding='utf-8') as f:
            for line in f:
                entry = json.loads(line)
                sentences.append(entry['text'])
                labels.append(entry['label'])

        print(f"📊 Đã tải {len(sentences)} mẫu.")

        print("\n⚙️ Đang mã hóa Vectors (sẽ mất chút thời gian)...")
        embeddings = embedder.encode(
            sentences,
            show_progress_bar=True,
            batch_size=32
        )

        # Chuẩn hóa L2 để dùng Cosine Similarity
        embeddings = embeddings.astype('float32')
        faiss.normalize_L2(embeddings)

        print("🗂️ Đang xây dựng FAISS Index...")
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatIP(dimension)  # Inner Product
        index.add(embeddings)

        print(f"✅ Đã index xong {index.ntotal} vectors.")

        # Lưu file
        print(f"\n💾 Đang lưu xuống {self.output_dir}...")
        faiss.write_index(index, str(self.index_path))

        metadata = {
            "labels": labels,
            "sentences": sentences,
            "model_name": self.model_name,
            "dimension": dimension,
            "total_samples": len(sentences)
        }

        with open(self.meta_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)

        print("🎉 BUILD THÀNH CÔNG!")

In [10]:
# ==============================================================================
# 2. ROUTER CLASS (Dùng để chạy Inference)
# ==============================================================================

class R1Router:
    """Router sử dụng Embedding để phân loại nhanh"""

    # Ngưỡng mặc định cho từng nhãn (Adaptive Thresholds)
    DEFAULT_THRESHOLDS = {
        'high_risk': 0.90,              # Cần chính xác cao
        'complex_consultation': 0.88,   # Cần tin cậy
        'emotional_support': 0.85,      # Trung bình
        'informational': 0.82           # Nới lỏng một chút
    }

    def __init__(
        self,
        model_dir: str = "../../models/router_r1",
        custom_thresholds: Optional[Dict[str, float]] = None
    ):
        self.model_dir = Path(model_dir)
        self.index_path = self.model_dir / "r1.index"
        self.meta_path = self.model_dir / "r1_metadata.json"

        self.thresholds = self.DEFAULT_THRESHOLDS.copy()
        if custom_thresholds:
            self.thresholds.update(custom_thresholds)

        self._load()

        # Warm up (chạy thử 1 câu để load model vào RAM)
        # print("🔥 Warming up model...")
        self.embedder.encode(["warmup"])

    def _load(self) -> None:
        if not self.meta_path.exists() or not self.index_path.exists():
            raise FileNotFoundError(f"Không tìm thấy file model R1 trong {self.model_dir}")

        with open(self.meta_path, 'r', encoding='utf-8') as f:
            self.meta = json.load(f)

        model_name = self.meta.get("model_name", "keepitreal/vietnamese-sbert")
        self.embedder = SentenceTransformer(model_name)

        try:
            self.index = faiss.read_index(str(self.index_path))
        except Exception as e:
            raise RuntimeError(f"Lỗi load FAISS index: {e}")

    def route(
        self,
        text: str,
        global_threshold: Optional[float] = None
    ) -> Tuple[Optional[str], float, Optional[str]]:
        """
        Phân loại 1 câu.
        Output: (label, confidence, matched_text)
        """
        vector = self.embedder.encode([text]).astype('float32')
        faiss.normalize_L2(vector)

        D, I = self.index.search(vector, k=1)

        score = float(D[0][0])
        idx = int(I[0][0])

        label = self.meta["labels"][idx]
        matched_text = self.meta["sentences"][idx]

        # Quyết định ngưỡng
        threshold = global_threshold or self.thresholds.get(label, 0.85)

        if score >= threshold:
            return label, score, matched_text

        return None, score, None

In [11]:
# ==============================================================================
# 3. BENCHMARK FUNCTION (Tự động Build & Test)
# ==============================================================================

def run_benchmark(test_file_path: str, train_file_path: str, model_dir: str):
    """
    Quy trình chuẩn:
    1. Check xem có model chưa -> Nếu chưa thì tự Build từ file Train.
    2. Load model.
    3. Chạy test trên file Test.
    """
    print(f"\n📊 KHỞI ĐỘNG QUY TRÌNH BENCHMARK...")
    model_path = Path(model_dir) / "r1.index"

    # --- BƯỚC 1: AUTO-BUILD (Nếu cần) ---
    if not model_path.exists():
        print(f"⚠️ Chưa thấy Model R1 tại {model_dir}")
        print(f"⚙️ Đang tự động Build từ file Train: {train_file_path}")

        if not os.path.exists(train_file_path):
            print(f"❌ LỖI: Không tìm thấy file Train ({train_file_path})")
            return

        try:
            builder = R1IndexBuilder(
                input_file=train_file_path,
                output_dir=model_dir
            )
            builder.build()
            print("✅ Auto-Build xong! Chuyển sang Test...")
        except Exception as e:
            print(f"❌ Lỗi khi Auto-Build: {e}")
            return
    else:
        print("✅ Đã tìm thấy Model R1. Đang load...")

    # --- BƯỚC 2: LOAD ROUTER ---
    try:
        router = R1Router(model_dir=model_dir)
    except Exception as e:
        print(f"❌ Lỗi load Router: {e}")
        return

    # --- BƯỚC 3: LOAD TEST DATA ---
    if not os.path.exists(test_file_path):
        print(f"❌ Không tìm thấy file test: {test_file_path}")
        return

    test_data = []
    with open(test_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            test_data.append(json.loads(line))

    print(f"🚀 Bắt đầu test trên {len(test_data)} mẫu...")

    # --- BƯỚC 4: CHẠY TEST ---
    hits = 0
    correct_hits = 0
    misses = 0
    TEST_THRESHOLD = 0.85

    print(f"⚙️ Ngưỡng tin cậy (Threshold): {TEST_THRESHOLD}")
    print("-" * 60)

    for sample in test_data:
        text = sample['text']
        true_label = sample['label']

        # Gọi Router
        pred_label, score, _ = router.route(text, global_threshold=TEST_THRESHOLD)

        if pred_label:
            hits += 1
            if pred_label == true_label:
                correct_hits += 1
            else:
                # In lỗi để debug (chỉ 3 lỗi đầu)
                if hits <= 3:
                    print(f"⚠️ Sai: '{text[:40]}...'")
                    print(f"   R1: {pred_label} ({score:.2f}) | Đúng: {true_label}")
        else:
            misses += 1

    # --- BƯỚC 5: BÁO CÁO ---
    total = len(test_data)
    hit_rate = (hits / total) * 100
    precision = (correct_hits / hits * 100) if hits > 0 else 0

    print("\n" + "="*60)
    print(f"🏆 KẾT QUẢ BENCHMARK R1 ROUTER")
    print("="*60)
    print(f"🔹 Tổng mẫu test:      {total}")
    print(f"⚡ R1 Tự xử lý (Hit):   {hits} ({hit_rate:.2f}%) -> Tiết kiệm {hits} lần gọi LLM")
    print(f"🤔 Chuyển cho LLM (Miss): {misses} ({100-hit_rate:.2f}%)")
    print("-" * 60)
    print(f"✅ Độ chính xác của R1 (Precision): {precision:.2f}%")
    print("="*60)

    if precision > 90:
        print("💡 Nhận xét: R1 hoạt động TUYỆT VỜI. Hệ thống sẽ rất nhanh.")
    elif precision > 80:
        print("💡 Nhận xét: R1 hoạt động TỐT.")
    else:
        print("💡 Nhận xét: Cần xem lại dữ liệu hoặc tăng ngưỡng.")

In [13]:
if __name__ == "__main__":
    # --- CẤU HÌNH ĐƯỜNG DẪN (CHỈNH SỬA Ở ĐÂY) ---
    BASE_PATH = "/content/drive/MyDrive/Major"

    # File V4 mới nhất của bạn
    TRAIN_FILE = f"{BASE_PATH}/Text_Reasoning/Text_Reasoning_train.jsonl"
    TEST_FILE  = f"{BASE_PATH}/Text_Reasoning/Text_Reasoning_test.jsonl"

    # Nơi lưu Model R1
    MODEL_DIR  = f"{BASE_PATH}/models/router_r1"

    # Chạy một lệnh duy nhất
    run_benchmark(test_file_path=TEST_FILE, train_file_path=TRAIN_FILE, model_dir=MODEL_DIR)


📊 KHỞI ĐỘNG QUY TRÌNH BENCHMARK...
⚠️ Chưa thấy Model R1 tại /content/drive/MyDrive/Major/models/router_r1
⚙️ Đang tự động Build từ file Train: /content/drive/MyDrive/Major/Text_Reasoning/Text_Reasoning_train.jsonl
🚀 Đang khởi tạo model embedding: keepitreal/vietnamese-sbert...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📂 Đang đọc dữ liệu từ /content/drive/MyDrive/Major/Text_Reasoning/Text_Reasoning_train.jsonl...
📊 Đã tải 5085 mẫu.

⚙️ Đang mã hóa Vectors (sẽ mất chút thời gian)...


Batches:   0%|          | 0/159 [00:00<?, ?it/s]

🗂️ Đang xây dựng FAISS Index...
✅ Đã index xong 5085 vectors.

💾 Đang lưu xuống /content/drive/MyDrive/Major/models/router_r1...
🎉 BUILD THÀNH CÔNG!
✅ Auto-Build xong! Chuyển sang Test...
🚀 Bắt đầu test trên 565 mẫu...
⚙️ Ngưỡng tin cậy (Threshold): 0.85
------------------------------------------------------------

🏆 KẾT QUẢ BENCHMARK R1 ROUTER
🔹 Tổng mẫu test:      565
⚡ R1 Tự xử lý (Hit):   405 (71.68%) -> Tiết kiệm 405 lần gọi LLM
🤔 Chuyển cho LLM (Miss): 160 (28.32%)
------------------------------------------------------------
✅ Độ chính xác của R1 (Precision): 100.00%
💡 Nhận xét: R1 hoạt động TUYỆT VỜI. Hệ thống sẽ rất nhanh.
